In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sympy import Symbol
from qrisp import *
from qrisp.quantum_backtracking import OHQInt
from zander_preparation_parametrized import *
from scipy.optimize import minimize


In [ ]:

def init_params_symb(graph, root, N):
    """
    This method returns a dictionary containing a list of abstract parameters for each node.

    Parameters
    ----------
    graph : networkx.DiGraph
        The directed graph representing the PBS.
    root : int
        The root of the graph.
    N : int
        The number sites.

    Returns
    -------
    params : dict
        A dictionary containing a list of abstract parameters for each node.
    symbols : list
        List of symbols associated to the abstract parameters

    """

    params = {}
    symbols=[Symbol("theta_"+str(root)+"_"+ str(i)) for i in range(N-1)]

    params[root] = [Symbol("theta_"+str(root)+"_"+ str(i)) for i in range(N-1)]

    recursive_init_params_symb(graph, root, N, params, symbols)

    return params,symbols

def recursive_init_params_symb(graph, node, N, params, symbols):
    """
    A recursive method for initializing a dictionary of abstract parameters for each node.

    Parameters
    ----------
    graph : networkx.DiGraph
        The directed graph representing the PBS.
    node : int
        The current node.
    N : int
        The number sites.
    q_array : QuantumArray
        The QuantumArray representing the PBS superposition state.
    params : dict
        A dictionary containing a list of angles for each node.

    """
    predecesors = list(graph.predecessors(node))
    m = len(predecesors)
    if(N<m+1):
        raise Exception(
                "Insufficient number of sites N"
        )

    for i in range(m):
        x=predecesors[i]
        params[predecesors[i]] = [ Symbol("theta_"+str(x)+"_"+ str(i))  for i in range(N-2-i)]

    for i in range(m):
      x=predecesors[i]
      for i in range(N-2-i):
        symbols.append(Symbol("theta_"+str(x)+"_"+ str(i)))


    # Recursivley add list of angles for predecessors
    for pred in predecesors:
        recursive_init_params_symb(graph, pred, N, params,symbols)

In [ ]:
def cost_symp(tot_coeff,n_parts,n_sites,Phi_graph):
    """
    returns the sympy cost function and the dictionary for the variables
    """
    combinations=list(tot_coeff[1].keys())

    x = {str(r)+str(i): Symbol(f"x{r}{i}") for r in range(n_parts) for i in range(n_sites)}
    
    cost=sum([ sum([ tot_coeff[r][(i,j)]*x[str(r)+str(i)]*x[str(s)+str(j)] for (i,j) in combinations]) for r,s in Phi_graph.edges()])
    
    return cost,x

# The following methods are used to move from the one hot encoding of the states
# to a binary one, in order to evaluate the cost function above.
# This can be done surely better with some qrisp feature and it will be changed.

def convert_to_binary(outcome_array):
    binary_string = ''
    for element in outcome_array:
        binary_string += ''.join('1' if i == element else '0' for i in range(max(outcome_array) + 1))
    return binary_string[::-1]


def translate_encoding(outdic):
    return {convert_to_binary(k):v  for k,v in outdic.items()}

In [ ]:
Phi = [(1,0),(2,0),(3,1)]

G = nx.DiGraph()
G.add_edges_from(Phi)
M = G.number_of_nodes() # Number of parts


N = 3
qtype = OHQInt(N)
q_array = QuantumArray(qtype = qtype, shape = (M))

params,symbols = init_params_symb(G, 0, N)

state = prepare_pbs_state(G, 0, N, q_array, params)
qc=state.qs.compile() #parameterized compiling


In [ ]:
# Cost coefficients from Table.3
cost_coeff={
    1: {(0,1): 1.64,
        (0,2): 1.05,
        (1,2): 0.59,
        },
    
    2: {(0,1): 5.56,
        (0,2): 3.54,
        (1,2): 1.98,
        },
 
     3: {(0,1): 8.06,
         (0,2): 5.14,
         (1,2): 2.88
         },
    }
tot_coeff = {}

for k,v in cost_coeff.items():
  tot_coeff[k]={}
  for rs,c in v.items():
    i,j=rs[0],rs[1]
    tot_coeff[k][(i,j)]=c
    tot_coeff[k][(j,i)]=c
    #avoid movements from a destination to the same
    tot_coeff[k][(i,i)]=10**30  #cost suggested by the challenge text
    tot_coeff[k][(j,j)]=10**30

In [ ]:
qubo,symb_dic=cost_symp(tot_coeff,M,N,G)
ord_symbs=list(symb_dic.values()) #ordering the symbols for the encoding

In [ ]:

def cost_function(res_dic,qubo,visualize=False):
    vars_ = ord_symbs
    b_dic=translate_encoding(res_dic)
    cost=0
    for k,v in b_dic.items():
      qval=qubo.subs({var:s for var, s in zip(vars_, k)})
      cost+=  qval * v
      if visualize:
        print(k,qval,v,qval * v)

    return cost



def optimization_wrapper(theta, qc, symbols, qarg, qubo):

    subs_dic = {symbols[i] : theta[i] for i in range(len(symbols))}

    res_dic = qarg.get_measurement(subs_dic = subs_dic, precompiled_qc = qc)
    return cost_function(res_dic,qubo)

In [ ]:
store_dicts={}
max_iter=100

optimal_state = [0, 2, 1, 0] #obtained with a brute force approach

# Running 10 times the optimization.
for exp in range(10):
  print('Experiment #',exp)

  # random initial point
  init_point=2*np.pi * np.random.rand(len(symbols))
  res_sample = minimize(optimization_wrapper,
                              init_point,
                              method='COBYLA',
                              options={'maxiter':max_iter},
                              args = (qc, symbols, q_array, qubo))  
    
  subs_dic = {s : res_sample.x[i] for i,s in enumerate(symbols)}
  res_dic = q_array.get_measurement(subs_dic = subs_dic, precompiled_qc = qc)
  best_result=list(res_dic.keys())[0]

  print('###  Best assignment: ###')
  print('   Cost:  ',qubo.subs({var:s for var, s in zip(ord_symbs, convert_to_binary(best_result))}))
  print('   State: ',best_result)
  print('   Prob:  ',list(res_dic.values())[0])
  store_dicts[exp]=res_dic